In [60]:
import pandas as pd

df_br = pd.read_csv('Dados/campeonato-brasileiro-full.csv')

df_br['data'] = pd.to_datetime(df_br['data'], dayfirst=True)

jogos_antes_hernanes = df_br[(df_br['data'].dt.year == 2017) & (df_br['rodata'] <= 16)]

jogos_depois_hernanes = df_br[(df_br['data'].dt.year == 2017) & (df_br['rodata'] >= 17)]

jogos_sao_paulo_antes_hernanes = jogos_antes_hernanes[(jogos_antes_hernanes['mandante'] == 'Sao Paulo') | (jogos_antes_hernanes['visitante'] == 'Sao Paulo')]

jogos_sao_paulo_depois_hernanes = jogos_depois_hernanes[(jogos_depois_hernanes['mandante'] == 'Sao Paulo') | (jogos_depois_hernanes['visitante'] == 'Sao Paulo')]

def determinar_vencedor(win):
    if win['mandante_Placar'] > win['visitante_Placar']:
        return win['mandante']
    elif win['mandante_Placar'] < win['visitante_Placar']:
        return win['visitante']
    else:
        return 'Empate'

jogos_sao_paulo_antes_hernanes['resultado'] = jogos_sao_paulo_antes_hernanes.apply(determinar_vencedor, axis=1)
jogos_sao_paulo_depois_hernanes['resultado'] = jogos_sao_paulo_depois_hernanes.apply(determinar_vencedor, axis=1)

def pontuação_sp(row):
    pontos_sao_paulo = 0
    if row['resultado'] == 'Sao Paulo':
        pontos_sao_paulo += 3
    elif row['resultado'] == 'Empate':
        pontos_sao_paulo += 1
    return pontos_sao_paulo

jogos_sao_paulo_antes_hernanes['pontos'] = jogos_sao_paulo_antes_hernanes.apply(pontuação_sp, axis=1)
jogos_sao_paulo_depois_hernanes['pontos'] = jogos_sao_paulo_depois_hernanes.apply(pontuação_sp, axis=1)

colunas_filtros = ['rodata', 'mandante', 'visitante', 'mandante_Placar', 'visitante_Placar', 'resultado', 'pontos']

jogos_sao_paulo_antes_hernanes = jogos_sao_paulo_antes_hernanes.loc[:, colunas_filtros]
jogos_sao_paulo_depois_hernanes = jogos_sao_paulo_depois_hernanes.loc[:, colunas_filtros]

#print("Jogos Antes de Hernanes:")
#print(jogos_sao_paulo_antes_hernanes.to_string(index=False))

#print("\nJogos Depois de Hernanes:")
#print(jogos_sao_paulo_depois_hernanes.to_string(index=False))


Jogos Antes de Hernanes:
 rodata     mandante   visitante  mandante_Placar  visitante_Placar    resultado  pontos
      1     Cruzeiro   Sao Paulo                1                 0     Cruzeiro       0
      2    Sao Paulo        Avai                2                 0    Sao Paulo       3
      3    Sao Paulo   Palmeiras                2                 0    Sao Paulo       3
      4  Ponte Preta   Sao Paulo                1                 0  Ponte Preta       0
      5    Sao Paulo     Vitoria                2                 0    Sao Paulo       3
      6  Corinthians   Sao Paulo                3                 2  Corinthians       0
      7        Sport   Sao Paulo                0                 0       Empate       1
      8    Sao Paulo Atletico-MG                1                 2  Atletico-MG       0
      9 Athletico-PR   Sao Paulo                1                 0 Athletico-PR       0
     10    Sao Paulo  Fluminense                1                 1       Empate     

C:\Users\ugoma\AppData\Local\Temp\ipykernel_16792\1561851982.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_sao_paulo_antes_hernanes['resultado'] = jogos_sao_paulo_antes_hernanes.apply(determinar_vencedor, axis=1)
C:\Users\ugoma\AppData\Local\Temp\ipykernel_16792\1561851982.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_sao_paulo_depois_hernanes['resultado'] = jogos_sao_paulo_depois_hernanes.apply(determinar_vencedor, axis=1)
C:\Users\ugoma\AppData\Local\Temp\ipykernel_16792\15618519